In [1]:
from catboost import  CatBoostClassifier
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import make_pipeline

import random
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

c:\프로그램\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
data = pd.read_csv('../datasets/BankChurners.csv')
list = ['Attrition_Flag', 'Total_Trans_Ct', 'Total_Trans_Amt', 'Total_Revolving_Bal', 'Total_Ct_Chng_Q4_Q1', 'Contacts_Count_12_mon', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Months_on_book']
data = data[list]
np.random.seed(42)
random.seed(42)

In [3]:
object_columns = data.select_dtypes('object').columns

for i in object_columns:

    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']



## Under Sampling

In [4]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]

for i in sampling:
    
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i == OneSidedSelection:
        sampling = i(random_state=42)
    else:
        sampling = i()

    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    cat = CatBoostClassifier(iterations=100, random_state=42, verbose=0)
    cat.fit(x_train_ss, y_train)
    pred = cat.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("Cat Model's score by Under sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

Cat Model's score by Under sampling


,f1_score,recall,precision,accuracy
TomekLinks,0.977830,0.986463,0.969346,0.962488
OneSidedSelection,0.977206,0.984108,0.970400,0.961500
NeighbourhoodCleaningRule,0.974661,0.973514,0.975811,0.957552
EditedNearestNeighbours,0.973117,0.969394,0.976868,0.955084
AllKNN,0.970868,0.961154,0.980781,0.951629
RepeatedEditedNearestNeighbours,0.969084,0.959388,0.978979,0.948667
NearMiss,0.930949,0.888758,0.977346,0.889437


## Over Sampling

In [5]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTE, BorderlineSMOTE, ADASYN]
strategy_ = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for i in sampling:
    
    for strategy in strategy_:
        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        ss = StandardScaler()
        x_train_ss = ss.fit_transform(x_train)
        x_test_ss = ss.fit_transform(x_test)

        sampling = i(sampling_strategy=strategy, random_state=42)
        x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

        cat = CatBoostClassifier(iterations=100, random_state=42, verbose=0)
        cat.fit(x_train_ss, y_train)
        pred = cat.predict(x_test_ss)

        acc = accuracy_score(y_test, pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        
        df.loc[i.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("Cat Model's score by Over sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Cat Model's score by Over sampling


,f1_score,recall,precision,accuracy
SMOTE(0.3),0.973816,0.974102,0.973529,0.956071
SMOTE(0.4),0.973069,0.967628,0.978571,0.955084
BorderlineSMOTE(0.3),0.971802,0.963508,0.980240,0.953110
ADASYN(0.3),0.969949,0.959388,0.980746,0.950148
BorderlineSMOTE(0.4),0.964854,0.945262,0.985276,0.942251
BorderlineSMOTE(0.5),0.963899,0.942908,0.985846,0.940770
SMOTE(0.6),0.960506,0.937610,0.984549,0.935341
SMOTE(0.5),0.959565,0.935845,0.984520,0.933860
SMOTE(0.7),0.954240,0.920541,0.990500,0.925962
ADASYN(0.4),0.951450,0.917010,0.988579,0.921520


## Combine Sampling

In [6]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTETomek, SMOTEENN]

for i in sampling:

    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i(random_state=42)
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    cat = CatBoostClassifier(iterations=100, random_state=42, verbose=0)
    cat.fit(x_train_ss, y_train)
    pred = cat.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[sampling] = [f1, recall, precision, acc]

print("Cat Model's score by Combine sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

Cat Model's score by Combine sampling


,f1_score,recall,precision,accuracy
SMOTETomek(random_state=42),0.935243,0.884049,0.992730,0.897335
SMOTEENN(random_state=42),0.934039,0.883461,0.990759,0.895360


## Pipeline을 통한 Combine Sampling

In [7]:
strategy_ = [0.3, 0.4, 0.5]
columns = ['f1_score', 'recall', 'precision', 'accuracy']

df = pd.DataFrame(columns=columns)

under_sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]
over_sampling = [SMOTE, BorderlineSMOTE, ADASYN]

cat = CatBoostClassifier(iterations=100, random_state=42, verbose=0)

for over in over_sampling:

    for under in under_sampling:

        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        for strategy in strategy_:
            
            if under == OneSidedSelection:
                under_ = under(random_state=42)
            else:
                under_ = under()

            over_ = over(sampling_strategy=strategy, random_state=42)

            ss = StandardScaler()
            x_train_ss = ss.fit_transform(x_train)
            x_test_ss = ss.fit_transform(x_test)

            model = make_pipeline(over_, under_, cat)   
            model.fit(x_train_ss, y_train)
            pred = model.predict(x_test_ss)

            acc = accuracy_score(y_test, pred)
            recall = recall_score(y_test, pred)
            precision = precision_score(y_test, pred)
            f1 = f1_score(y_test, pred)

            df.loc[over.__name__+under.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("Cat Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Cat Model's score by sampling


,f1_score,recall,precision,accuracy
SMOTEOneSidedSelection(0.3),0.974646,0.972925,0.976373,0.957552
SMOTETomekLinks(0.3),0.974646,0.972925,0.976373,0.957552
SMOTENeighbourhoodCleaningRule(0.3),0.973551,0.964097,0.983193,0.956071
SMOTEOneSidedSelection(0.4),0.972428,0.965274,0.979689,0.954097
ADASYNTomekLinks(0.3),0.971768,0.962331,0.981393,0.953110
SMOTETomekLinks(0.4),0.971564,0.965274,0.977937,0.952616
BorderlineSMOTENeighbourhoodCleaningRule(0.3),0.970816,0.959388,0.982520,0.951629
BorderlineSMOTETomekLinks(0.3),0.970291,0.961154,0.979604,0.950642
BorderlineSMOTEOneSidedSelection(0.3),0.969932,0.958799,0.981325,0.950148
SMOTENearMiss(0.4),0.969860,0.956445,0.983656,0.950148


## Under, Over, Combine Sampling과 Pipeline을 통한 Combine Sampling 전체 결과

In [8]:
sampling = [EditedNearestNeighbours,RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]

for i in sampling:
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i in [OneSidedSelection, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]:
        sampling = i(random_state=42)
    else:
        sampling = i()
        
    x_train, y_train = sampling.fit_resample(x_train, y_train)

    cat = CatBoostClassifier(iterations=100, random_state=42, verbose=0)
    cat.fit(x_train, y_train)
    pred = cat.predict(x_test)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    df.loc[i.__name__] = [f1, recall, precision, acc]

print("Cat Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Cat Model's score by sampling


,f1_score,recall,precision,accuracy
TomekLinks,0.978274,0.980577,0.975981,0.963475
OneSidedSelection,0.977153,0.981754,0.972595,0.961500
SMOTEOneSidedSelection(0.3),0.974646,0.972925,0.976373,0.957552
SMOTETomekLinks(0.3),0.974646,0.972925,0.976373,0.957552
SMOTENeighbourhoodCleaningRule(0.3),0.973551,0.964097,0.983193,0.956071
NeighbourhoodCleaningRule,0.972877,0.971160,0.974601,0.954590
SMOTEOneSidedSelection(0.4),0.972428,0.965274,0.979689,0.954097
ADASYNTomekLinks(0.3),0.971768,0.962331,0.981393,0.953110
SMOTETomekLinks(0.4),0.971564,0.965274,0.977937,0.952616
EditedNearestNeighbours,0.971377,0.958799,0.984290,0.952616


## No Sampling

In [9]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
No_Sampling = pd.DataFrame(columns=columns)

x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.fit_transform(x_test)

## CatBoost는 cat_features에 문자열 인덱스를 넣어주면 알아서 라벨링해주는 기능이 있다.
cat = CatBoostClassifier(iterations=100, random_state=42,verbose=0)
cat.fit(x_train, y_train) #, cat_features=object_col
pred = cat.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

No_Sampling.loc['No Sampling'] = [f1, recall, precision, acc]

print("Cat Model's score by sampling")
No_Sampling.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
No_Sampling.head(10)

Cat Model's score by sampling


,f1_score,recall,precision,accuracy
No Sampling,0.976107,0.985874,0.966532,0.959526
